In [1]:
%%capture
import warnings
warnings.filterwarnings('ignore')

import calitp_data_analysis.magics

from rt_analysis import rt_filter_map_plot
import build_speedmaps_index

from IPython.display import display, Markdown
import pandas as pd
import geopandas as gpd
from siuba import *

import shared_utils
import segment_speed_utils

In [2]:
catalog = shared_utils.catalog_utils.get_catalog('gtfs_analytics_data')

In [36]:
## parameters cell
itp_id = 182

In [37]:
analysis_date = build_speedmaps_index.ANALYSIS_DATE
import datetime as dt
# if itp_id == 293:
#     analysis_date = dt.date(2023, 9, 13)

In [38]:
speedmap_index = pd.read_parquet(f'_rt_progress_{analysis_date}.parquet') >> filter(_.organization_itp_id == itp_id)
# speedmap_index = pd.read_parquet(f'_rt_progress_{analysis_date}.parquet') >> filter(_.organization_name.str.contains('Torr'))

In [39]:
speedmap_index

,organization_name,organization_itp_id,organization_source_record_id,caltrans_district,_is_current,analysis_date,schedule_gtfs_dataset_key,status
42,Los Angeles County Metropolitan Transportation...,182,recPnGkwdpnr8jmHB,07 - Los Angeles,True,2024-10-16,0666caf3ec1ecc96b74f4477ee4bc939,speedmap_segs_available


In [40]:
path = f'{catalog.speedmap_segments.dir}{catalog.speedmap_segments.shape_stop_single_segment_detail}_{analysis_date}.parquet'

In [41]:
speedmap_segs = gpd.read_parquet(path) #  aggregated

In [42]:
# speedmap_segs.columns

In [43]:
speedmap_segs = speedmap_segs >> filter(_.schedule_gtfs_dataset_key == speedmap_index.schedule_gtfs_dataset_key.iloc[0])

In [69]:
speedmap_segs >> head(3)

,schedule_gtfs_dataset_key,shape_array_key,shape_id,route_id,route_short_name,stop_pair,stop_pair_name,segment_id,time_of_day,p50_mph,...,p20_mph,p80_mph,name,caltrans_district,organization_source_record_id,organization_name,base64_url,geometry,n_trips_sch,trips_hr_sch
9729,0666caf3ec1ecc96b74f4477ee4bc939,1d801329ec1c7b1da3460ede3c0fe8bc,1200106_JUNE24,120-13183,120,16644__14345,Whittwood Center__Santa Gertrudes / Starbuck,16644-14345-1,AM Peak,14.49,...,13.00,16.38,LA Metro Bus Schedule,07 - Los Angeles,recPnGkwdpnr8jmHB,Los Angeles County Metropolitan Transportation...,aHR0cHM6Ly9naXRsYWIuY29tL0xBQ01UQS9ndGZzX2J1cy...,"LINESTRING (-117.99579 33.94440, -117.99579 33...",4,1.333333
9730,0666caf3ec1ecc96b74f4477ee4bc939,1d801329ec1c7b1da3460ede3c0fe8bc,1200106_JUNE24,120-13183,120,14345__14346,Santa Gertrudes / Starbuck__Santa Gertrudes / ...,14345-14346-1,AM Peak,26.67,...,22.58,28.96,LA Metro Bus Schedule,07 - Los Angeles,recPnGkwdpnr8jmHB,Los Angeles County Metropolitan Transportation...,aHR0cHM6Ly9naXRsYWIuY29tL0xBQ01UQS9ndGZzX2J1cy...,"LINESTRING (-117.99307 33.93974, -117.99307 33...",4,1.333333
9731,0666caf3ec1ecc96b74f4477ee4bc939,1d801329ec1c7b1da3460ede3c0fe8bc,1200106_JUNE24,120-13183,120,14346__14344,Santa Gertrudes / Yermo__Santa Gertrudes / Lef...,14346-14344-1,AM Peak,11.10,...,7.31,16.04,LA Metro Bus Schedule,07 - Los Angeles,recPnGkwdpnr8jmHB,Los Angeles County Metropolitan Transportation...,aHR0cHM6Ly9naXRsYWIuY29tL0xBQ01UQS9ndGZzX2J1cy...,"LINESTRING (-117.99308 33.93630, -117.99308 33...",4,1.333333


In [45]:
peak_test = speedmap_segs >> filter(_.time_of_day == 'AM Peak')

In [46]:
def prep_gdf(gdf):
    
    gdf = gdf.to_crs(calitp_data_analysis.geography_utils.CA_NAD83Albers)

    ## shift to right side of road to display direction
    gdf.geometry = gdf.geometry.apply(shared_utils.rt_utils.try_parallel)
    gdf = gdf.apply(shared_utils.rt_utils.arrowize_by_frequency, axis=1, frequency_col='trips_hr_sch')

    gdf = gdf >> arrange(_.trips_hr_sch)

    return gdf

In [47]:
peak_test = prep_gdf(peak_test)

In [48]:
cmap = shared_utils.rt_utils.ACCESS_ZERO_THIRTY_COLORSCALE
url = shared_utils.rt_utils.ACCESS_SPEEDMAP_LEGEND_URL

In [49]:
peak_test >> head(2)

,schedule_gtfs_dataset_key,shape_array_key,shape_id,route_id,route_short_name,stop_pair,stop_pair_name,segment_id,time_of_day,p50_mph,...,p20_mph,p80_mph,name,caltrans_district,organization_source_record_id,organization_name,base64_url,geometry,n_trips_sch,trips_hr_sch
34253,0666caf3ec1ecc96b74f4477ee4bc939,4901c1a3b0efda4363bafac83036f8af,1110423_JUNE24,111-13183,111,12569__14726,Studebaker / Borson__Studebaker / Imperial,12569-14726-1,AM Peak,8.59,...,8.59,8.59,LA Metro Bus Schedule,07 - Los Angeles,recPnGkwdpnr8jmHB,Los Angeles County Metropolitan Transportation...,aHR0cHM6Ly9naXRsYWIuY29tL0xBQ01UQS9ndGZzX2J1cy...,"POLYGON ((175823.879 -453844.437, 175808.618 -...",1,0.333333
34254,0666caf3ec1ecc96b74f4477ee4bc939,4901c1a3b0efda4363bafac83036f8af,1110423_JUNE24,111-13183,111,14726__14724,Studebaker / Imperial__Studebaker / Firestone,14726-14724-1,AM Peak,7.14,...,7.14,7.14,LA Metro Bus Schedule,07 - Los Angeles,recPnGkwdpnr8jmHB,Los Angeles County Metropolitan Transportation...,aHR0cHM6Ly9naXRsYWIuY29tL0xBQ01UQS9ndGZzX2J1cy...,"POLYGON ((175820.029 -453597.893, 175804.674 -...",1,0.333333


In [50]:
peak_test.columns

Index(['schedule_gtfs_dataset_key', 'shape_array_key', 'shape_id', 'route_id',
       'route_short_name', 'stop_pair', 'stop_pair_name', 'segment_id',
       'time_of_day', 'p50_mph', 'n_trips', 'p20_mph', 'p80_mph', 'name',
       'caltrans_district', 'organization_source_record_id',
       'organization_name', 'base64_url', 'geometry', 'n_trips_sch',
       'trips_hr_sch'],
      dtype='object')

## export map

In [58]:
speedmap_state = shared_utils.rt_utils.set_state_export(
                                    peak_test, filename=f'{itp_id}_new', map_type='speedmap',
                                    color_col='p20_mph', cmap=cmap, legend_url=url,
                                    cache_seconds=0, map_title=f'Speedmap Segs {time_of_day} {analysis_date}')

writing to calitp-map-tiles/testing/182_new.geojson.gz


In [59]:
speedmap_state

{'state_dict': {'name': 'null',
  'layers': [{'name': 'Speedmap Segs Early AM 2024-10-16',
    'url': 'https://storage.googleapis.com/calitp-map-tiles/testing/182_new.geojson.gz',
    'properties': {'stroked': False,
     'highlight_saturation_multiplier': 0.5,
     'tooltip_speed_key': 'p20_mph'},
    'type': 'speedmap'}],
  'lat_lon': (34.05429096594501, -118.29944262320139),
  'zoom': 13,
  'legend_url': 'https://storage.googleapis.com/calitp-map-tiles/speeds_legend_color_access.svg'},
 'spa_link': 'https://embeddable-maps.calitp.org/?state=eyJuYW1lIjogIm51bGwiLCAibGF5ZXJzIjogW3sibmFtZSI6ICJTcGVlZG1hcCBTZWdzIEVhcmx5IEFNIDIwMjQtMTAtMTYiLCAidXJsIjogImh0dHBzOi8vc3RvcmFnZS5nb29nbGVhcGlzLmNvbS9jYWxpdHAtbWFwLXRpbGVzL3Rlc3RpbmcvMTgyX25ldy5nZW9qc29uLmd6IiwgInByb3BlcnRpZXMiOiB7InN0cm9rZWQiOiBmYWxzZSwgImhpZ2hsaWdodF9zYXR1cmF0aW9uX211bHRpcGxpZXIiOiAwLjUsICJ0b29sdGlwX3NwZWVkX2tleSI6ICJwMjBfbXBoIn0sICJ0eXBlIjogInNwZWVkbWFwIn1dLCAibGF0X2xvbiI6IFszNC4wNTQyOTA5NjU5NDUwMSwgLTExOC4yOTk0NDI2MjMyMDEzOV

## other test

In [60]:
time_of_day = 'PM Peak'

In [61]:
period_speeds = speedmap_segs >> filter(_.time_of_day == time_of_day)

In [62]:
period_test = prep_gdf(period_speeds)

In [63]:
time_of_day_lower = time_of_day.lower().replace(' ', '_')

In [65]:
speedmap_state = shared_utils.rt_utils.set_state_export(
                                    period_test, filename=f'{itp_id}_{time_of_day_lower}_new', map_type='speedmap',
                                    color_col='p20_mph', cmap=cmap, legend_url=url,
                                    cache_seconds=0, map_title=f'Speedmap Segs {time_of_day} {analysis_date}')

writing to calitp-map-tiles/testing/182_pm_peak_new.geojson.gz


In [66]:
speedmap_state

{'state_dict': {'name': 'null',
  'layers': [{'name': 'Speedmap Segs PM Peak 2024-10-16',
    'url': 'https://storage.googleapis.com/calitp-map-tiles/testing/182_pm_peak_new.geojson.gz',
    'properties': {'stroked': False,
     'highlight_saturation_multiplier': 0.5,
     'tooltip_speed_key': 'p20_mph'},
    'type': 'speedmap'}],
  'lat_lon': (34.04804074656423, -118.29635317134563),
  'zoom': 13,
  'legend_url': 'https://storage.googleapis.com/calitp-map-tiles/speeds_legend_color_access.svg'},
 'spa_link': 'https://embeddable-maps.calitp.org/?state=eyJuYW1lIjogIm51bGwiLCAibGF5ZXJzIjogW3sibmFtZSI6ICJTcGVlZG1hcCBTZWdzIFBNIFBlYWsgMjAyNC0xMC0xNiIsICJ1cmwiOiAiaHR0cHM6Ly9zdG9yYWdlLmdvb2dsZWFwaXMuY29tL2NhbGl0cC1tYXAtdGlsZXMvdGVzdGluZy8xODJfcG1fcGVha19uZXcuZ2VvanNvbi5neiIsICJwcm9wZXJ0aWVzIjogeyJzdHJva2VkIjogZmFsc2UsICJoaWdobGlnaHRfc2F0dXJhdGlvbl9tdWx0aXBsaWVyIjogMC41LCAidG9vbHRpcF9zcGVlZF9rZXkiOiAicDIwX21waCJ9LCAidHlwZSI6ICJzcGVlZG1hcCJ9XSwgImxhdF9sb24iOiBbMzQuMDQ4MDQwNzQ2NTY0MjMsIC0xMTguMjk